In [1]:
import numpy as np
import torch
import sys
from utils import *
from matplotlib import pyplot as plt
import torch.nn as nn
import time
from numba import cuda
from tqdm import tqdm, trange
import os
import pyroc
import pandas as pd
import gc
from IPython.display import clear_output
sys.path.append("..") 
import global_config

num_models = global_config.test_param_configs['num_models']
num_repeats = global_config.test_param_configs['num_repeats']
ns = global_config.test_param_configs['n_tr_list']
n_te = global_config.test_param_configs['n_te']
n_ev = global_config.test_param_configs['n_ev']

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"  
device = torch.device("cuda:0")
dtype = torch.float32
torch.manual_seed(42)
np.random.seed(42)

/math/home/eruisun/software/anaconda/envs/LFI/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = np.load('HIGGS.npy')
dataset_P = dataset[dataset[:,0]==0][:, 1:] # background (5829122, 28)
dataset_Q = dataset[dataset[:,0]==1][:, 1:] # signal     (5170877, 28)
dataset_P = MatConvert(dataset_P, device=device, dtype=dtype)
dataset_Q = MatConvert(dataset_Q, device=device, dtype=dtype)

# Define function

In [3]:
def generate_PQ(X_test, Y_test, model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst,
                n_test, Samples, batch_size = 5000, If_n_large_MonteCarlo = 1000, test_batch_size=20000):
    '''
    Input: 
        n_eval: n_eval in our paper, then we generate $X^{eval}$ and $Y^{eval}$ used to compute MMD.
        Samples: we generate 'Samples' samples from P and Q, then compute their scores f(sample;X_eval,Y_eval)
    Output:
        X_eval, Y_eval: generate from P and Q, each with size (n_eval, 28)
        P_scores, Q_scores: scores of samples from P and Q, each with size (Samples,)
        EKxx, EKyy, EKxy: expectation of K(X,X), K(Y,Y), K(X,Y), where X~P_X, Y~P_Y. Then the gamma in our paper can be computed from these three values.
    '''
    # X_test = dataset_P[np.random.choice(dataset_P.shape[0], n_test, replace=False)]
    # Y_test = dataset_Q[np.random.choice(dataset_Q.shape[0], n_test, replace=False)]
    EKxx, EKyy, EKxy = compute_gamma(X_test, Y_test, model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst, MonteCarlo=If_n_large_MonteCarlo)
    batches = (Samples-1)//batch_size + 1
    P_scores = np.zeros(Samples)
    Q_scores = np.zeros(Samples)
    for i in trange(batches):
        remain = batch_size
        if i==batches-1:
            remain = Samples - batch_size*(batches-1)
        S_hand = np.concatenate((dataset_P[np.random.choice(dataset_P.shape[0], remain, replace=False)],
                    dataset_Q[np.random.choice(dataset_Q.shape[0], remain, replace=False)]), axis=0)
        S_hand = MatConvert(S_hand, device, dtype)
        PQhat_hand = compute_score_func(S_hand, X_test, Y_test, 
                    model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst,
                    M = test_batch_size)
        PQhat_hand = PQhat_hand.cpu().detach().numpy()
        P_scores[i*batch_size: i*batch_size+remain] = PQhat_hand[:remain]
        Q_scores[i*batch_size: i*batch_size+remain] = PQhat_hand[remain:]
        del S_hand, PQhat_hand
        gc.collect()
    return X_test, Y_test, P_scores, Q_scores, EKxx, EKyy, EKxy

class PQ_data():
    '''
    Define a class to compute the p-value, the Type I/II error.
    '''
    def __init__(self, P_scores, Q_scores, EKxx, EKyy, EKxy):
        self.P_scores = P_scores
        self.Q_scores = Q_scores
        self.P_mean = np.mean(P_scores)
        self.P_std = np.std(P_scores)
        self.Q_mean = np.mean(Q_scores)
        self.Q_std = np.std(Q_scores)
        self.EKxx = EKxx
        self.EKyy = EKyy
        self.EKxy = EKxy
    def pval_T_m_in_sigma(self, pi, m, use_gaussian, MonteCarlo):
        '''
        Compute the expected significance of discovery.
        Input:
            pi: the strength of the mixture in our paper.
            m: the number of samples, i.e. the size of Z in our paper.
            use_gaussian: if use Gaussian Approximation, then use_gaussian = True, else use_gaussian = False.
            MonteCarlo: when use_gaussian = False, we use Monte Carlo to approximate the expectation.
        Output:
            p: the expected significance of discovery (in units of Gaussian sigma)
        '''
        T = pi*self.Q_mean + (1-pi)*self.P_mean
        P_scores = self.P_scores
        mean = self.P_mean
        std = self.P_std
        if m==1:
            p = np.mean(P_scores > T)
            p = -scipy.stats.norm.ppf(p)
            self.p = p
            return p
        if use_gaussian:
            p = (T-mean)/std*np.sqrt(m)
        else:
            T_mix_MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                idx = np.random.choice(P_scores.shape[0], m, replace=False)
                T_mix_MonteCarlo_list[i] = np.mean(P_scores[idx])
            p = np.mean(T_mix_MonteCarlo_list > T)
            p = -scipy.stats.norm.ppf(p)
        self.p = p
        return p
    def type_1_error_H0(self, pi, m, use_gaussian, MonteCarlo):
        '''
        Compute the estimated Type I error.
        Input:
            pi: the strength of the mixture in our paper.
            m: the number of samples, i.e. the size of Z in our paper.
            use_gaussian: if use Gaussian Approximation, then use_gaussian = True, else use_gaussian = False.
            MonteCarlo: when use_gaussian = False, we use Monte Carlo to approximate the expectation.
        Output:
            type_1_error: the estimated Type I error.   
        '''
        P_scores = self.P_scores
        Q_scores = self.Q_scores
        mean = self.P_mean
        std = self.P_std
        P_mean = self.P_mean
        P_std = self.P_std
        Q_mean = self.Q_mean
        gamma = self.EKxx*(pi/2-1) + self.EKxy*(1-pi) + self.EKyy*(pi/2)
        #gamma = (pi/2)*Q_mean + (1-pi/2)*P_mean
        self.gamma = gamma
        if m==1:
            type_1_error = np.mean(P_scores > gamma)
            self.type_1_error = type_1_error
            return type_1_error
        if use_gaussian:
            type_1_error = 1-scipy.stats.norm.cdf((gamma-mean)/std*np.sqrt(m))
        else:
            MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                idx = np.random.choice(P_scores.shape[0], m, replace=False)
                MonteCarlo_list[i] = np.mean(P_scores[idx])
            type_1_error = np.mean(MonteCarlo_list > gamma)
            del MonteCarlo_list
            gc.collect()
        self.type_1_error = type_1_error
        return type_1_error
    def type_2_error_H1(self, pi, m, use_gaussian, MonteCarlo):
        '''
        Compute the estimated Type II error.
        Input:
            pi: the strength of the mixture in our paper.
            m: the number of samples, i.e. the size of Z in our paper.
            use_gaussian: if use Gaussian Approximation, then use_gaussian = True, else use_gaussian = False.
            MonteCarlo: when use_gaussian = False, we use Monte Carlo to approximate the expectation.
        Output:
            type_2_error: the estimated Type II error.              
        '''
        P_scores = self.P_scores
        Q_scores = self.Q_scores
        P_mean = self.P_mean
        P_std = self.P_std
        Q_mean = self.Q_mean
        Q_std = self.Q_std
        gamma = self.EKxx*(pi/2-1) + self.EKxy*(1-pi) + self.EKyy*(pi/2)
        #gamma = (pi/2)*Q_mean + (1-pi/2)*P_mean
        self.gamma = gamma
        if m==1:
            type_2_error = np.mean(Q_scores < gamma)
            self.type_2_error = type_2_error
            return type_2_error
        if use_gaussian:
            mean = Q_mean*pi + P_mean*(1-pi)
            std = np.sqrt(pi*Q_std**2 + (1-pi)*P_std**2 + pi*(1-pi)*(P_mean-Q_mean)**2)
            type_2_error = scipy.stats.norm.cdf((gamma-mean)/std*np.sqrt(m))
        else:
            MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                Signals_idx = np.random.choice(Q_scores.shape[0], int(m*pi), replace=False)
                Backgrounds_idx = np.random.choice(P_scores.shape[0], int(m*(1-pi)), replace=False)
                MonteCarlo_list[i] = np.mean(np.concatenate((Q_scores[Signals_idx], P_scores[Backgrounds_idx])))
            type_2_error = np.mean(MonteCarlo_list < gamma)
            del MonteCarlo_list
            gc.collect()
        self.type_2_error = type_2_error
        return type_2_error

# Start to calculate p for different n

In [4]:
# ns is list of n_train
repeats = num_models
ns, repeats

([50000, 5000], 5)

In [5]:
for n in ns:
    try:
        ps_L, ps_L_thres = get_p_from_10_model(n, 'Res_Net', repeats)
        np.save('./Res_Net/%d_soft.npy'%n, ps_L)
        np.save('./Res_Net/%d_hard.npy'%n, ps_L_thres)
        # clear_output(wait=True)
    except: print('fail')

fail
fail


In [6]:
for n in ns:
    try:
        ps_L, ps_L_thres = get_p_from_10_model(n, 'Mix', repeats)
        np.save('./Mix/%d_soft.npy'%n, ps_L)
        np.save('./Mix/%d_hard.npy'%n, ps_L_thres)
        clear_output(wait=True)
    except: print('fail')

fail
fail


In [7]:
for n in ns:
    try:
        ps_B, ps_B_thres = get_p_from_10_model(n, 'LBI', repeats)
        np.save('./LBI/%d_soft.npy'%n, ps_B)
        clear_output(wait=True)
    except: print('fail')

fail
fail


In [8]:
for n in ns:
    try:
        ps_G, ps_G_thres = get_p_from_10_model(n, 'Fea_Gau', repeats)
        np.save('./Fea_Gau/%d_soft.npy'%n, ps_G)
        np.save('./Fea_Gau/%d_hard.npy'%n, ps_G_thres)
        clear_output(wait=True)
    except: print('fail')

fail
fail


In [9]:
for n in ns:
    try:
        ps_O, ps_O_thres = get_p_from_10_model(n, 'Gaussian', repeats)
        np.save('./Gaussian/%d_soft.npy'%n, ps_O)
        np.save('./Gaussian/%d_hard.npy'%n, ps_O_thres)
        clear_output(wait=True)
    except: print('fail')

fail
fail


In [10]:
for n in ns:
    try:
        ps_S, ps_S_thres = get_p_from_10_model(n, 'Scheffe', repeats)
        np.save('./Scheffe/%d_soft.npy'%n, ps_S)
        np.save('./Scheffe/%d_hard.npy'%n, ps_S_thres)
        _, ps_S_thres_05 = get_p_from_10_model(n, 'Scheffe', repeats, force_thres=0.5)
        np.save('./Scheffe/%d_05.npy'%n, ps_S_thres_05)
        clear_output(wait=True)
    except: print('fail')

fail
fail


In [11]:
try:
    !python UME/Higgs_kmod_test.py
    clear_output(wait=True)
except: print('fail')

------------------- n_tr = 50000 -------------------
  0%|                                                     | 0/5 [00:24<?, ?it/s]
Traceback (most recent call last):
  File "/math/home/eruisun/github/LFI/UME/Higgs_kmod_test.py", line 172, in <module>
    run_test()
  File "/math/home/eruisun/github/LFI/UME/Higgs_kmod_test.py", line 152, in run_test
    p_soft_mat[i,:], p_hard_mat[i,:] = simulate_p_value(n_tr_label, n_ev, min(n_tr,n_te),
  File "/math/home/eruisun/github/LFI/UME/Higgs_kmod_test.py", line 81, in simulate_p_value
    thres, _, _ = get_thres_from_evaluated_scores(X_test_scores, Y_test_scores)
  File "/math/home/eruisun/github/LFI/UME/utils_UME.py", line 319, in get_thres_from_evaluated_scores
    auc, x, y = get_auc_from_evaluated_scores(X,Y)
  File "/math/home/eruisun/github/LFI/UME/utils_UME.py", line 312, in get_auc_from_evaluated_scores
    fpr, tpr = roc._roc(pred) # False positive rate, True positive rate
  File "/math/home/eruisun/software/anaconda/envs/LFI/lib/p

In [ ]:
try:
    !python RFM/Higgs_RFM_test.py
    !python RFM/Higgs_RFM_used_in_MMD.py
    clear_output(wait=True)
except: print('fail')

------------------- n_tr = 5000 -------------------
100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.60s/it]
------------------- n_tr = 5000 -------------------
------------------- n_tr = 5000 -------------------
100%|█████████████████████████████████████████████| 1/1 [00:07<00:00,  7.31s/it]


# Packge the pval data into one file

In [ ]:
def load_pval(ns):
    method_ntr_pval_dict = {
        'Res_Net_soft':{},
        'Res_Net_hard':{},
        'Mix_soft':{},
        'Mix_hard':{},
        'LBI_soft':{},
        # 'LBI_hard':{},
        'Fea_Gau_soft':{},
        'Fea_Gau_hard':{},
        'Gaussian_soft':{},
        'Gaussian_hard':{},
        'Scheffe_soft':{},
        'Scheffe_hard':{},
        'Scheffe_05':{},
        'UME_soft':{},
        'UME_hard':{}
    }
    for n in ns:
        method_ntr_pval_dict['Res_Net_soft'][n] = np.load('./Res_Net/%d_soft.npy'%n)
        method_ntr_pval_dict['Res_Net_hard'][n] = np.load('./Res_Net/%d_hard.npy'%n)
        method_ntr_pval_dict['Mix_soft'][n] = np.load('./Mix/%d_soft.npy'%n)
        method_ntr_pval_dict['Mix_hard'][n] = np.load('./Mix/%d_hard.npy'%n)
        method_ntr_pval_dict['LBI_soft'][n] = np.load('./LBI/%d_soft.npy'%n)
        # method_ntr_pval_dict['LBI_hard'][n] = np.load('./LBI/%d_hard.npy'%n)
        method_ntr_pval_dict['Fea_Gau_soft'][n] = np.load('./Fea_Gau/%d_soft.npy'%n)
        method_ntr_pval_dict['Fea_Gau_hard'][n] = np.load('./Fea_Gau/%d_hard.npy'%n)
        method_ntr_pval_dict['Gaussian_soft'][n] = np.load('./Gaussian/%d_soft.npy'%n)
        method_ntr_pval_dict['Gaussian_hard'][n] = np.load('./Gaussian/%d_hard.npy'%n)
        method_ntr_pval_dict['Scheffe_soft'][n] = np.load('./Scheffe/%d_soft.npy'%n)
        method_ntr_pval_dict['Scheffe_hard'][n] = np.load('./Scheffe/%d_hard.npy'%n)
        method_ntr_pval_dict['Scheffe_05'][n] = np.load('./Scheffe/%d_05.npy'%n)
        method_ntr_pval_dict['UME_soft'][n] = np.load('./UME/pval_data/n_tr=%d_soft.npy'%n)
        method_ntr_pval_dict['UME_hard'][n] = np.load('./UME/pval_data/n_tr=%d_hard.npy'%n)
    return method_ntr_pval_dict


In [ ]:
import numpy as np
# ns = np.array([1300000, 1000000, 700000, 400000, 200000, 50000])
method_ntr_pval_dict = load_pval(ns)

FileNotFoundError: [Errno 2] No such file or directory: './Mix/5000_soft.npy'

In [ ]:
import pickle
with open('./pval_dict.pkl', 'wb') as f:
        pickle.dump(method_ntr_pval_dict, f, pickle.HIGHEST_PROTOCOL)